In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from datetime import timedelta, datetime
import requests
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
import pandas_ta as ta  # Replaced talib with pandas_ta
import time

# **Hyperparameters**
lookback = 10  # Lookback period for LSTM
epochs = 200   # Number of training epochs
batch_size = 8  # Batch size for training
learning_rate = 0.0001  # Learning rate for optimizer

# **Fetch Data**
def fetch_and_preprocess_data():
    def pivotid(df1, l, n1, n2): 
        if (l-n1 < 0 or l+n2 >= len(df1)):
            return 0
        
        pividlow = 1
        pividhigh = 1
        for i in range(l-n1, l+n2+1):
            if df1.Low[l] > df1.Low[i]:
                pividlow = 0
            if df1.High[l] < df1.High[i]:
                pividhigh = 0
        if pividlow and pividhigh:
            return 3
        elif pividlow:
            return 1
        elif pividhigh:
            return 2
        else:
            return 0

    def pointpos(x):
        if x['pivot'] == 1:
            return x['Low']
        elif x['pivot'] == 2:
            return x['High']
        else:
            return 0
    end_time = int(datetime.now().timestamp() * 1000)
    start_time = int((datetime.now() - timedelta(days=365)).timestamp() * 1000)

    url = f"https://public.coindcx.com/market_data/candles?pair=I-ETH_INR&interval=1m&startTime={start_time}&endTime={end_time}"
    
    response = requests.get(url)
    data = response.json()

    btc_data = pd.DataFrame(data)
    btc_data = btc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'volume': 'Volume', 'close': 'Close'})

    btc_data['time'] = pd.to_datetime(btc_data['time'], unit='ms')
    btc_data[['Open', 'High', 'Low', 'Close', 'Volume']] = btc_data[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

    # Add Technical Indicators using pandas_ta
    btc_data['rsi'] = ta.rsi(btc_data['Close'], length=15)
    macd = ta.macd(btc_data['Close'], fast=12, slow=26, signal=9)
    btc_data['macd'] = macd['MACD_12_26_9']
    btc_data['macd_signal'] = macd['MACDs_12_26_9']
    btc_data['macd_hist'] = macd['MACDh_12_26_9']
    bbands = ta.bbands(btc_data['Close'], length=20)
    btc_data['upperband'] = bbands['BBU_20_2.0']
    btc_data['middleband'] = bbands['BBM_20_2.0']
    btc_data['lowerband'] = bbands['BBL_20_2.0']

    btc_data['EMAF'] = ta.ema(btc_data['Close'], length=20)
    btc_data['EMAM'] = ta.ema(btc_data['Close'], length=100)
    btc_data['EMAS'] = ta.ema(btc_data['Close'], length=150)

    btc_data['ema_50'] = ta.ema(btc_data['Close'], length=50)
    btc_data['ema_200'] = ta.ema(btc_data['Close'], length=200)

    btc_data.fillna(btc_data.mean(), inplace=True)

    # **Target Variable: Next Close**
    btc_data['Next_Close'] = btc_data['Close'].shift(-1)

    # btc_data['pivot'] = btc_data.apply(lambda x: pivotid(btc_data, x.name, 10, 10), axis=1)
    # btc_data['pointpos'] = btc_data.apply(lambda x: pointpos(x), axis=1)
    
    btc_data.dropna(inplace=True)

    # **Scaling**
    feature_cols = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    btc_data[feature_cols] = scaler_x.fit_transform(btc_data[feature_cols])
    btc_data['Next_Close'] = scaler_y.fit_transform(btc_data[['Next_Close']])

    return btc_data, scaler_x, scaler_y

# **LSTM Dataset Creation**
def create_lstm_dataset(data, lookback=30):
    X, y = [], []
    feature_cols = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
    for i in range(lookback, len(data)):
        X.append(data[feature_cols].iloc[i-lookback:i].values)
        y.append(data['Next_Close'].iloc[i])
    return np.array(X), np.array(y)

# **LSTM Model**
def create_lstm_model(input_shape):
    model = Sequential()
    
    model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.001)), input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001))))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001))))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Bidirectional(LSTM(32, return_sequences=False, kernel_regularizer=l2(0.001))))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))

    return model

# **Labeling Function**
def generate_labels(close_diff):
    """
    Generate labels based on the difference between next price and current price.
    Label 1 (Buy Signal): close_diff > 0 (next price > current price).
    Label 2 (Sell Signal): close_diff < 0 (next price < current price).
    Label 0 (Hold Signal): close_diff == 0 (next price == current price).
    """
    labels = np.zeros_like(close_diff)  # Initialize with Hold signals (0)
    labels[close_diff > 0] = 1  # Buy signals (1)
    labels[close_diff < 0] = 2  # Sell signals (2)
    return labels

# **Training Loop**
while True:
    btc_data, scaler_x, scaler_y = fetch_and_preprocess_data()
    if btc_data is None or btc_data.empty:
        print("No data available. Retrying in 10 seconds...")
        time.sleep(10)
        continue

    train_data, test_data = train_test_split(btc_data, test_size=0.2, shuffle=False)

    X_train, y_train = create_lstm_dataset(train_data, lookback=lookback)
    X_test, y_test = create_lstm_dataset(test_data, lookback=lookback)

    # Check for empty datasets
    if len(X_train) == 0 or len(X_test) == 0:
        print("Not enough data for training. Retrying...")
        time.sleep(10)
        continue

    # **Train LSTM**
    lstm_model = create_lstm_model((lookback, X_train.shape[2]))
    optimizer = Adam(learning_rate=learning_rate)
    lstm_model.compile(optimizer=optimizer, loss='mse')

    # **Callbacks**
    early_stopping = EarlyStopping(
        monitor='val_loss',  # Monitor validation loss
        patience=10,         # Stop training if no improvement for 10 epochs
        restore_best_weights=True  # Restore the best model weights
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',  # Monitor validation loss
        factor=0.1,          # Reduce learning rate by 50%
        patience=20,          # Wait for 5 epochs without improvement
        min_lr=0.0001       # Minimum learning rate
    )

    print("Training LSTM model...")
    history = lstm_model.fit(
        X_train, 
        y_train, 
        epochs=epochs, 
        batch_size=batch_size, 
        validation_split=0.3,
        callbacks=[reduce_lr]
    )

    # **Predictions & De-scaling**
    y_pred = lstm_model.predict(X_test)
    y_test_descaled = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_descaled = scaler_y.inverse_transform(y_pred).flatten()

    # **Calculate close_diff**
    close_diff = y_test_descaled - y_test_descaled  # For actual labels
    predicted_close_diff = y_pred_descaled - y_test_descaled  # For predicted labels

    # **Generate Labels**
    y_test_labels = generate_labels(close_diff)  # Actual labels
    y_pred_labels = generate_labels(predicted_close_diff)  # Predicted labels

    # **Check Label Distribution**
    print("\nActual Labels Distribution:")
    print(pd.Series(y_test_labels).value_counts())

    print("\nPredicted Labels Distribution:")
    print(pd.Series(y_pred_labels).value_counts())

    # **Check Model Predictions**
    print("\nSample Actual Values (Descaled):", y_test_descaled[:10])
    print("Sample Predicted Values (Descaled):", y_pred_descaled[:10])

    # **Error Calculation on Descaled Values**
    mse = mean_squared_error(y_test_descaled, y_pred_descaled)
    mae = mean_absolute_error(y_test_descaled, y_pred_descaled)
    mape = mean_absolute_percentage_error(y_test_descaled, y_pred_descaled)
    rmse = np.sqrt(mse)

    print("\nLSTM Test Metrics (Descaled):")
    print(f"MSE: {mse:.6f}")  # Mean Squared Error (in original scale)
    print(f"MAE: {mae:.6f}")  # Mean Absolute Error (in original scale)
    print(f"MAPE: {mape:.4f}%")  # Mean Absolute Percentage Error (in original scale)
    print(f"RMSE: {rmse:.6f}")  # Root Mean Squared Error (in original scale)
    requests.post("http://localhost:5000/update", json={"LSTM": mae})
    # **Save Model**
    lstm_model.save('lstm_model_coindcx_optimized.h5')
    print("\nModel saved as 'lstm_model_coindcx_optimized.h5'.")
    requests.post("http://localhost:5000/update", json={"Updated": "LSTM"})
    print("\nModel training done.")
      # Exit the loop after successful training

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training LSTM model...
Epoch 1/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - loss: 2.5555 - val_loss: 2.0903 - learning_rate: 1.0000e-04
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 2.2921 - val_loss: 2.0485 - learning_rate: 1.0000e-04
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 2.2875 - val_loss: 2.0024 - learning_rate: 1.0000e-04
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 2.2345 - val_loss: 1.9563 - learning_rate: 1.0000e-04
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 2.1288 - val_loss: 1.9052 - learning_rate: 1.0000e-04
Epoch 6/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - loss: 2.1855 - val_loss: 1.8913 - learning_rate: 1.0000e-04
Epoch 7/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 2.0814 - val_loss: 1.9020 - learning_rate: 1.0000e-04
Epoch 8/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 2.0852 - val_loss: 1.9348 - learning_rate: 1.0000e-04
Epoch 9/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 2.036


Actual Labels Distribution:
0.0    90
Name: count, dtype: int64

Predicted Labels Distribution:
2.0    76
1.0    14
Name: count, dtype: int64

Sample Actual Values (Descaled): [217990.   218000.   218000.   218000.   218000.   218000.   217979.99
 218000.   216622.55 217862.63]
Sample Predicted Values (Descaled): [216952.69 216919.08 216902.11 216856.33 216739.48 216665.1  216599.06
 216544.73 216499.42 216467.88]

LSTM Test Metrics (Descaled):
MSE: 2165890.267494
MAE: 1260.056632
MAPE: 0.0058%
RMSE: 1471.696391

Model saved as 'lstm_model_coindcx_optimized.h5'.

Model training done.
Training LSTM model...
Epoch 1/200


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - loss: 3.1402 - val_loss: 2.1451 - learning_rate: 1.0000e-04
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 2.3873 - val_loss: 2.1169 - learning_rate: 1.0000e-04
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 2.1696 - val_loss: 2.0811 - learning_rate: 1.0000e-04
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 2.1356 - val_loss: 2.0762 - learning_rate: 1.0000e-04
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 2.1486 - val_loss: 2.1844 - learning_rate: 1.0000e-04
Epoch 6/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 2.0777 - val_loss: 2.1945 - learning_rate: 1.0000e-04
Epoch 7/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 2.0478 - val_loss: 2.2195 - learning_rate: 1.0000e-04
Epoch 8/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 2.0161 - val_loss: 2.1315 - learning_rate: 1.0000e-04
Epoch 9/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 2.0346 - val_loss: 2.0970 - learning_rat

...

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 2.6665 - val_loss: 2.0295 - learning_rate: 1.0000e-04
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 2.2933 - val_loss: 2.0348 - learning_rate: 1.0000e-04
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.3706 - val_loss: 2.0736 - learning_rate: 1.0000e-04
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 2.2025 - val_loss: 2.1462 - learning_rate: 1.0000e-04
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 2.1191 - val_loss: 2.1689 - learning_rate: 1.0000e-04
Epoch 6/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 2.2490 - val_loss: 2.1671 - learning_rate: 1.0000e-04
Epoch 7/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 2.0714 - val_loss: 2.1346 - learning_rate: 1.0000e-04
Epoch 8/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 2.0747 - val_loss: 2.0457 - learning_rate: 1.0000e-04
Epoch 9/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 2.0705 - val_loss: 1.9981 - learning_rat


Actual Labels Distribution:
0.0    90
Name: count, dtype: int64

Predicted Labels Distribution:
2.0    61
1.0    29
Name: count, dtype: int64

Sample Actual Values (Descaled): [217252.65 217340.31 217616.79 217473.39 217625.74 217479.44 216000.41
 217344.44 217750.83 217820.05]
Sample Predicted Values (Descaled): [216167.9  216196.58 216467.   216599.77 217015.12 217661.2  217938.33
 217550.62 217146.33 217303.  ]

LSTM Test Metrics (Descaled):
MSE: 680849.133939
MAE: 740.977868
MAPE: 0.0034%
RMSE: 825.135828

Model saved as 'lstm_model_coindcx_optimized.h5'.

Model training done.
Training LSTM model...
Epoch 1/200


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 2.4119 - val_loss: 1.9695 - learning_rate: 1.0000e-04
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.1918 - val_loss: 1.9793 - learning_rate: 1.0000e-04
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 2.2475 - val_loss: 1.9874 - learning_rate: 1.0000e-04
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 2.1457 - val_loss: 1.9439 - learning_rate: 1.0000e-04
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 2.2242 - val_loss: 1.9253 - learning_rate: 1.0000e-04
Epoch 6/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 2.0727 - val_loss: 1.9016 - learning_rate: 1.0000e-04
Epoch 7/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 2.1189 - val_loss: 1.8602 - learning_rate: 1.0000e-04
Epoch 8/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 604s 18s/step - loss: 2.0981 - val_loss: 1.8443 - learning_rate: 1.0000e-04
Epoch 9/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 2.0455 - val_loss: 1.8564 - learning_ra